In [1]:
import math
from collections import namedtuple
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from copy import deepcopy
from tqdm import tqdm
from tqdm.notebook import tqdm
import random

import numba as nb
# from numba import jit, vectorize, float64

In [2]:
with open('./data/tsp_51_1', 'r') as input_data_file:
    input_data = input_data_file.read()
    
lines = input_data.split('\n')
nodeCount = int(lines[0])

points = []
for i in range(1, nodeCount+1):
    line = lines[i]
    parts = line.split()
    # points.append(Point(float(parts[0]), float(parts[1])))
    points.append((float(parts[0]), float(parts[1])))
points = np.array(points)

In [3]:
@nb.jit(nopython=True)
def length_distance(single_point, all_points):
    return np.sqrt((all_points[:, 0]-single_point[0])**2 + (all_points[:, 1]-single_point[1])**2)


@nb.jit(nopython=True)
def calculate_travel(graph):
    n = len(graph)
    total_distance = 0.0

    for i in range(n - 1):
        total_distance += DISTANCE_MATRIX[graph[i], graph[i + 1]]

    # Add the distance to return to the starting point
    total_distance += DISTANCE_MATRIX[graph[-1], graph[0]]

    return total_distance



In [4]:
def init_path(node_count):
    ## Keep picking the next nearest neighbor to get the path.
    pick_next = 0
    exploring_path = np.array([], dtype=int)
    exploring_path = np.append(exploring_path, pick_next)
    
    while exploring_path.size < nodeCount:
        
        neighbor_idx = np.argsort(DISTANCE_MATRIX[pick_next])
        
        mask = np.isin(neighbor_idx, exploring_path)
        neighbor_idx = neighbor_idx[~mask]
    
        pick_next = neighbor_idx[0]
        
        exploring_path = np.append(exploring_path, pick_next)

    total_distance = calculate_travel(exploring_path)

    return exploring_path, total_distance

In [5]:
DISTANCE_MATRIX = np.zeros((nodeCount, nodeCount))
for i in tqdm(range(len(points))):
    DISTANCE_MATRIX[i] = length_distance(points[i], points)

  0%|          | 0/51 [00:00<?, ?it/s]

In [6]:
class Penalty:
    def __init__(self, node_count):
        self.penalty = [[0] * i for i in range(1, node_count + 1)]

    def __getitem__(self, key):
        j, i = sorted(key)
        return self.penalty[i][j]

    def __setitem__(self, key, value):
        j, i = sorted(key)
        self.penalty[i][j] = value

In [7]:
class Activate:
    def __init__(self, size):
        self.flag = [1] * size
        self.count_ = size

    def set_1(self, i):
        if self.flag[i] == 0:
            self.count_ += 1
        self.flag[i] = 1

    def set_0(self, i):
        if self.flag[i] == 1:
            self.count_ -= 1
        self.flag[i] = 0

    def get(self, i):
        return self.flag[i]

    def size(self):
        return len(self.flag)

In [180]:
# @nb.jit(nopython=True)
def do_2opt(pointidx1, pointidx2, path, new_distance, augmented_distance):
    path = deepcopy(path)
    if pointidx2 > pointidx1:
        slice_start = pointidx1
        slice_end = pointidx2
    else:
        slice_start = pointidx2
        slice_end = pointidx1

    slice_path = path[slice_start:slice_end+1]
    slice_path = np.flip(slice_path)

    new_path = np.hstack((path[:slice_start], slice_path, path[slice_end + 1:]))

    point1 = current_path[pointidx1-1]
    point2 = current_path[pointidx1]
    point3 = current_path[pointidx2]
    point4 = current_path[pointidx2+1] if pointidx2+1 < nodeCount else current_path[0]

    d12 = DISTANCE_MATRIX[point1][point2]
    p12 = PENALTY[point1, point2]
    
    d34 = DISTANCE_MATRIX[point3][point4]
    d13 = DISTANCE_MATRIX[point1][point3]
    d24 = DISTANCE_MATRIX[point2][point4]
    p34 = PENALTY[point3, point4]
    p13 = PENALTY[point1, point3]
    p24 = PENALTY[point2, point4]

    # net_distance = d13 + d24 - d12 - d34
    # augmented_distance_delta =  net_distance + lambda_factor * (p13 + p24 - p12 - p34)

    net_distance = d12 + d34 - d13 - d24
    augmented_distance_delta =  (d13 + lambda_factor * p13) + (d24 + lambda_factor * p24) - (d12 + lambda_factor * p12) - (d34 + lambda_factor * p34)

    new_distance -= net_distance
    augmented_distance -= augmented_distance_delta 
    
    return new_path, new_distance, augmented_distance


# def pick_next_edge(pointidx1, current_path):
def pick_next_edge(point2, path):
    path = deepcopy(path)
    # point1 = current_path[pointidx1-1]
    # point2 = current_path[pointidx1]

    pointidx1 = np.where(path==point2)[0][0]
    point1 = path[pointidx1-1]
    
    d12 = DISTANCE_MATRIX[point1][point2]
    p12 = PENALTY[point1, point2]
    
    
    # Repeatedly do 2-opt for all the neighbors
    next_probable_edge = {}
    max_distance_delta = float('inf')
    
    node = path[pointidx1]
    sorted_neighbor_points = np.argsort(DISTANCE_MATRIX[node])[1:]
    
    for neighbor_idx in sorted_neighbor_points:
       
        pointidx2 = np.where(path==neighbor_idx)[0][0]
        
        point3 = path[pointidx2]
        point4 = path[pointidx2+1] if pointidx2+1 < nodeCount else path[0]
    
    
        d34 = DISTANCE_MATRIX[point3][point4]
        d13 = DISTANCE_MATRIX[point1][point3]
        d24 = DISTANCE_MATRIX[point2][point4]
        
        p34 = PENALTY[point3, point4]
        p13 = PENALTY[point1, point3]
        p24 = PENALTY[point2, point4]

        # augmented_distance_delta = (d13 + d24 - d12 - d34) + lambda_factor * (p13 + p24 - p12 - p34)
        augmented_distance_delta = (d13 + lambda_factor * p13) + (d24 + lambda_factor * p24) - (d12 + lambda_factor * p12) - (d34 + lambda_factor * p34)
        
        next_probable_edge[pointidx2] = augmented_distance_delta

    # filter candidates with max value
    max_distance_delta = max(next_probable_edge.values())
    print('delta: ', max_distance_delta)
    print(next_probable_edge)
    viable_candidates = [key for key, value in next_probable_edge.items() if value == max_distance_delta]

    # Pick random viable candidate
    print('candidates: ', len(viable_candidates))
    if max_distance_delta > 1e-6:
        pointidx2 = random.choice(viable_candidates)
        point3 = path[pointidx2]
        point4 = path[pointidx2+1] if pointidx2+1 < nodeCount else path[0]
        print(point1, point2, point3, point4)
    else:
        # not using None as index could be 0. if None and if 0 has same effect.
        pointidx2 = -1
        point3 = -1
        point4 = -1
    
    return (pointidx1, pointidx2), (point1, point2, point3, point4)

In [55]:
step_limit = 1000000
SEARCH_AREA = nodeCount//5
PENALTY = Penalty(nodeCount)
ACTIVATE = Activate(nodeCount)

current_path, current_distance = init_path(nodeCount)

alpha = 0.1
lambda_factor = 0.0 #alpha * current_distance/nodeCount

In [39]:
lambda_factor

1.1116539124168678

In [ ]:
current_path = np.array([0, 33, 5, 2, 28, 45, 9, 10, 3, 46, 24, 34, 23, 12, 36, 6, 26, 47, 27, 41, 8, 4, 35, 13, 7, 19, 40, 18, 11, 42, 29, 43, 21, 37, 20, 25, 1, 22, 31, 39, 50, 38, 15, 44, 16, 14, 30, 48, 32, 17, 49])
current_distance = calculate_travel(current_path)
current_augmented_distance = current_distance

In [ ]:
indexes, points_visited = pick_next_edge(3, current_path)
indexes

In [ ]:
current_path, current_distance, current_augmented_distance = do_2opt(3, 18, current_path, current_distance, current_distance)

In [ ]:
current_distance, current_augmented_distance

In [ ]:
indexes, points_visited = pick_next_edge(3, current_path)

In [ ]:
ACTIVATE.get(0)

In [ ]:
ACTIVATE.count_

In [181]:
step_limit = 1000000
SEARCH_AREA = nodeCount//5
PENALTY = Penalty(nodeCount)
ACTIVATE = Activate(nodeCount)

current_path, current_distance = init_path(nodeCount)

alpha = 0.1
lambda_factor = 0.0 #lpha * current_distance/nodeCount

In [182]:
current_path = np.array([0, 33, 5, 2, 28, 45, 9, 10, 3, 46, 24, 34, 23, 12, 36, 6, 26, 47, 27, 41, 8, 4, 35, 13, 7, 19, 40, 18, 11, 42, 29, 43, 21, 37, 20, 25, 1, 22, 31, 39, 50, 38, 15, 44, 16, 14, 30, 48, 32, 17, 49])
current_distance = calculate_travel(current_path)
current_augmented_distance = current_distance

optimal_distance = current_distance
optimal_path = deepcopy(current_path)

In [183]:
while ACTIVATE.count_ > 0:
    for explore_node in range(ACTIVATE.size()):
        
        if not ACTIVATE.get(explore_node):
            continue
    
        indexes, points_visited = pick_next_edge(explore_node, current_path)
        
        pt_idx1, pt_idx2 = indexes
        p1, p2, p3, p4 = points_visited

        if pt_idx2 == -1:
            ACTIVATE.set_0(p3)
            print('Setting zero: ', explore_node)
            continue
        
        current_path, current_distance, current_augmented_distance = do_2opt(pt_idx1, pt_idx2, current_path, current_distance, current_augmented_distance)
        print(current_path)
        
        ACTIVATE.set_1(p1)
        ACTIVATE.set_1(p2)
        ACTIVATE.set_1(p3)
        ACTIVATE.set_1(p4)
        
        if current_distance < optimal_distance:
            optimal_path = deepcopy(current_path)
            optimal_distance = current_distance
    break

delta:  85.27284111282941
{1: 1.0707188750484669, 2: 18.482652152739416, 48: -0.3077290969909434, 37: 5.3192495627260215, 47: -10.307926207491107, 3: 29.070525654410293, 17: 27.964933752893334, 16: 15.690522729124353, 36: 2.0718162491727306, 38: 8.809935479055945, 49: -3.552713678800501e-15, 15: 18.53983211862326, 50: -49.678969393496885, 4: 44.58005923299421, 35: 12.444191891691574, 18: 44.52286012607116, 34: 28.379391134321576, 39: 15.58323343472761, 14: 29.089275588727673, 5: 52.81076673950339, 6: 60.200621917510965, 7: 48.0325675082191, 40: 24.344376415837136, 32: 31.630865352614606, 33: 27.996826738766742, 31: 24.177566351221543, 19: 57.268574201276934, 8: 61.70961628144468, 13: 40.05774793671786, 46: -6.574943569126084, 10: 62.80445123169295, 30: 32.58093377034295, 11: 61.86184692341182, 9: 64.71835705191108, 29: 44.793951556194614, 12: 51.78224961457505, 28: 53.48024121425586, 41: 46.65140980846965, 20: 76.15545351537476, 21: 74.61576949128846, 22: 80.08117519470227, 26: 67.7696

In [ ]:
optimal_distance, current_augmented_distance

In [ ]:
indexes, points_visited = pick_next_edge(3, current_path)
pt_idx1, pt_idx2 = indexes
current_path, current_distance, current_augmented_distance = do_2opt(pt_idx1, pt_idx2, current_path, current_distance, current_distance)

In [121]:
ACTIVATE.count_

50

In [159]:
temp = {31: -12.649110640673518, 33: 18.31990028169172, 6: 15.759871289201275, 30: 0.0, 29: 9.51421774985615, 7: 26.179715294287924, 34: 28.846289127772025, 5: 16.418982067430683, 35: 33.40235365597169, 36: 0.8118388698140109, 8: 36.31327297118845, 37: 32.90239917888687, 16: 37.77803642838529, 15: 18.39565835594091, 9: 47.47260513758245, 48: 40.632700223783885, 4: 35.86986724389317, 47: 38.08534827019248, 38: 50.40737105962688, 17: 51.355462395498705, 18: 54.01818951928887, 14: 47.468517988503876, 10: 60.307376469620294, 13: 54.481420071828545, 12: 59.71415587871449, 19: 64.51611161595473, 46: 25.45823547227819, 50: 18.688863334454773, 49: 51.77326281353311, 3: 52.41409839083403, 11: 64.89330178595058, 20: 67.31235672698354, 39: 70.29675675348348, 2: 72.39543639800213, 21: 74.25282923191997, 40: 80.30101649596851, 24: 87.26091711579821, 1: 77.17591894895801, 23: 81.97577363743162, 22: 79.1169833034654, 26: 97.11685237860478, 25: 93.52264641454795, 0: 88.67543374783126, 41: 98.55560878554905, 28: 8.356379388383218, 27: 102.78822602356635, 42: 109.04103382674626, 44: 110.1819634748281, 43: 107.15590726281751, 45: 59.01984002607658}

In [161]:
max(temp.values())

110.1819634748281

In [163]:
temp

{31: -12.649110640673518,
 33: 18.31990028169172,
 6: 15.759871289201275,
 30: 0.0,
 29: 9.51421774985615,
 7: 26.179715294287924,
 34: 28.846289127772025,
 5: 16.418982067430683,
 35: 33.40235365597169,
 36: 0.8118388698140109,
 8: 36.31327297118845,
 37: 32.90239917888687,
 16: 37.77803642838529,
 15: 18.39565835594091,
 9: 47.47260513758245,
 48: 40.632700223783885,
 4: 35.86986724389317,
 47: 38.08534827019248,
 38: 50.40737105962688,
 17: 51.355462395498705,
 18: 54.01818951928887,
 14: 47.468517988503876,
 10: 60.307376469620294,
 13: 54.481420071828545,
 12: 59.71415587871449,
 19: 64.51611161595473,
 46: 25.45823547227819,
 50: 18.688863334454773,
 49: 51.77326281353311,
 3: 52.41409839083403,
 11: 64.89330178595058,
 20: 67.31235672698354,
 39: 70.29675675348348,
 2: 72.39543639800213,
 21: 74.25282923191997,
 40: 80.30101649596851,
 24: 87.26091711579821,
 1: 77.17591894895801,
 23: 81.97577363743162,
 22: 79.1169833034654,
 26: 97.11685237860478,
 25: 93.52264641454795,
 0: 